In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

# function to train-test-split data and treat it

def split_and_treat_data(X, y, save_scaler=False, randomstate=None):
    # splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=randomstate)

    # transforming numericals
    transformer = MinMaxScaler().fit(X_train)

    X_train = pd.DataFrame(transformer.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(transformer.transform(X_test), columns=X.columns)


    if save_scaler:
        pickle.dump(transformer, open('scaler.sav', 'wb'))

    return X_train, X_test, y_train, y_test, transformer

In [3]:
from sklearn.metrics import mean_squared_error

# function to fit a model to data

def fit_to_data(model, X_train, y_train, X_test, y_test):
    model_name = str(model).split('(')[0]
    print(model_name, 'stats:')
    
    model.fit(X_train, y_train)

    test_predictions = model.predict(X_test)

    print('score >>>', model.score(X_test, y_test))
    print('rmse  >>>', mean_squared_error(y_test, test_predictions)**0.5)

    return model

In [4]:
def predict_data(model, X, scaler):
    nums = pd.DataFrame(scaler.transform(X), columns=X.columns)

    predictions = pd.DataFrame(model.predict(nums))
    predictions = predictions.apply(lambda x: round(x, 2))

    return predictions  

In [5]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

In [6]:
def clean_state_hol(x):
    if x == 'a':
        return 1
    if x == 'b':
        return 2
    if x == 'c':
        return 3
    else:
        return x

In [7]:
data = pd.read_csv('sales.csv')

data['year'] = data['date'].apply(lambda x: x.split('-')[0])
data['month'] = data['date'].apply(lambda x: x.split('-')[1])
data['day_of_month'] = data['date'].apply(lambda x: x.split('-')[2])
# data['day_of_week'] = data['day_of_week'].astype(object)
# data['school_holiday'] = data['school_holiday'].astype(object)
# data['open'] = data['open'].astype(object)
# data['promotion'] = data['promotion'].astype(object)

# data['state_holiday'] = pd.factorize(data['state_holiday'])[0]
data['state_holiday'] = data['state_holiday'].apply(clean_state_hol)

data.columns = data.columns.str.lower()

data_open = data[data['open'] == 1].copy()
data_open = data_open.drop(['open'], axis=1) # only include data in the model, where the store was actually open (no sales otherwise)

y = data['sales']
y_open = data_open['sales']

data = data.drop(['unnamed: 0', 'date', 'sales'], axis=1)
data_open = data_open.drop(['unnamed: 0', 'date', 'sales'], axis=1)
data = data.astype(int)
data_open = data_open.astype(int)
data

,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day_of_month
0,366,4,517,1,0,0,0,2013,4,18
1,394,6,694,1,0,0,0,2015,4,11
2,807,4,970,1,1,0,0,2013,8,29
3,802,2,473,1,1,0,0,2013,5,28
4,726,4,1068,1,1,0,0,2013,10,10
...,...,...,...,...,...,...,...,...,...,...
640835,409,6,483,1,0,0,0,2013,10,26
640836,97,1,987,1,1,0,0,2014,4,14
640837,987,1,925,1,0,0,0,2014,7,7
640838,1084,4,725,1,0,0,0,2014,6,12


In [8]:
data.dtypes

store_id               int64
day_of_week            int64
nb_customers_on_day    int64
open                   int64
promotion              int64
state_holiday          int64
school_holiday         int64
year                   int64
month                  int64
day_of_month           int64
dtype: object

In [9]:
validation = pd.read_csv('validation_for_students.csv')
true_ind = validation['True_index']
validation = validation.drop(['True_index'], axis=1)
validation.columns = [col.lower() for col in validation.columns]

validation['year'] = validation['date'].apply(lambda x: x.split('-')[0])
validation['month'] = validation['date'].apply(lambda x: x.split('-')[1])
validation['day_of_month'] = validation['date'].apply(lambda x: x.split('-')[2])
# validation['day_of_week'] = validation['day_of_week'].astype(object)
# validation['school_holiday'] = validation['school_holiday'].astype(object)
# validation['open'] = validation['open'].astype(object)
# validation['promotion'] = validation['promotion'].astype(object)

# validation['state_holiday'] = pd.factorize(validation['state_holiday'])[0]
validation['state_holiday'] = validation['state_holiday'].apply(clean_state_hol)

validation = validation.drop(['date'], axis=1)
validation = validation.astype(int)
validation

,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day_of_month
0,764,4,0,0,0,3,1,2013,12,26
1,22,3,449,1,0,0,1,2013,5,22
2,1087,6,622,1,0,0,0,2013,6,29
3,139,6,314,1,0,0,0,2013,8,17
4,568,1,356,1,0,0,0,2014,4,7
...,...,...,...,...,...,...,...,...,...,...
71200,217,2,633,1,1,0,0,2015,1,13
71201,604,3,743,1,1,0,0,2014,4,30
71202,1021,5,1852,1,1,0,1,2014,7,18
71203,28,3,0,0,0,0,1,2014,8,27


In [10]:
features = ['nb_customers_on_day', 'store_id', 'promotion', 'day_of_month', 'day_of_week', 'state_holiday', 'school_holiday', 'year', 'month']

In [11]:
X_train, X_test, y_train, y_test, sca = split_and_treat_data(data[features], y, save_scaler=True)
lr = fit_to_data(XGBRegressor(), X_train, y_train, X_test, y_test)

XGBRegressor stats:
score >>> 0.9632084188911652
rmse  >>> 743.2872500189997


In [12]:
def clean_closed(row):
    if row['open'] == 0:
        return 0
    else:
        return row[0]

def clean_neg(x):
    if float(x) < 0:
        return 0
    else:
        return x

In [13]:
prediction_df = predict_data(lr, data[features], scaler=sca)
compare = pd.concat([data, y.reset_index(drop=True), prediction_df], axis=1)
compare[0] = compare.apply(lambda row: clean_closed(row), axis=1)
compare[0] = compare[0].apply(clean_neg)
compare.head(30)

,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day_of_month,sales,0
0,366,4,517,1,0,0,0,2013,4,18,4422,4491.339844
1,394,6,694,1,0,0,0,2015,4,11,8297,7776.919922
2,807,4,970,1,1,0,0,2013,8,29,9729,9976.129883
3,802,2,473,1,1,0,0,2013,5,28,6513,5633.109863
4,726,4,1068,1,1,0,0,2013,10,10,10882,9426.599609
5,674,2,1016,1,1,0,0,2013,3,19,8406,8970.860352
6,659,7,0,0,0,0,0,2014,6,8,0,0.000000
7,27,3,1106,1,1,0,1,2014,7,16,11162,10152.500000
8,347,6,448,1,0,0,0,2013,5,11,5559,4854.740234
9,288,2,291,1,0,0,0,2013,6,25,3997,3299.949951


In [14]:
mean_squared_error(compare['sales'], compare[0])**0.5

730.6787441524785

In [15]:
prediction_df = predict_data(lr, validation[features], scaler=sca)
compare = pd.concat([validation, prediction_df], axis=1)
compare[0] = compare.apply(lambda row: clean_closed(row), axis=1)
compare[0] = compare[0].apply(clean_neg)
compare

,store_id,day_of_week,nb_customers_on_day,open,promotion,state_holiday,school_holiday,year,month,day_of_month,0
0,764,4,0,0,0,3,1,2013,12,26,0.000000
1,22,3,449,1,0,0,1,2013,5,22,4035.000000
2,1087,6,622,1,0,0,0,2013,6,29,6214.930176
3,139,6,314,1,0,0,0,2013,8,17,3460.070068
4,568,1,356,1,0,0,0,2014,4,7,3512.500000
...,...,...,...,...,...,...,...,...,...,...,...
71200,217,2,633,1,1,0,0,2015,1,13,5573.140137
71201,604,3,743,1,1,0,0,2014,4,30,8488.669922
71202,1021,5,1852,1,1,0,1,2014,7,18,15711.129883
71203,28,3,0,0,0,0,1,2014,8,27,0.000000


In [16]:
pd.concat([true_ind, compare[0]], axis=1).to_csv('awesome.csv', index=False)